In [1]:
from datasets import load_dataset
import openai
import os
import itertools
import logging
import sys

import dill as pickle
import dspy
from dsp.utils import deduplicate
from dspy.teleprompt import BootstrapFewShot, LabeledFewShot, BayesianSignatureOptimizer, BootstrapFewShotWithRandomSearch
from dspy.evaluate import answer_exact_match, answer_passage_match
from dspy.evaluate.evaluate import Evaluate
from dsp.utils import deduplicate
from dspy.primitives import module

from copy import copy
import random
import json
import tqdm
import pickle
import pandas as pd
from pandas import json_normalize
import numpy as np

import models

random.seed(1)

In [2]:
from dotenv import load_dotenv

load_dotenv() 

True

## LM and RM

In [3]:
# setting lm and rm in dspy
openai_key = os.environ["OPENAI_API_KEY"]
colbert_server = 'http://index.contextual.ai:8893/api/search'

lm = dspy.OpenAI(model='gpt-3.5-turbo', api_key=openai_key)
rm = dspy.ColBERTv2(url=colbert_server)
dspy.settings.configure(lm=lm, rm=rm)

## Dataset

In [4]:
# dataset download and split
def get_squad_split(squad, split="validation"):
    data = zip(*[squad[split][field] for field in squad[split].features])
    exs = [dspy.Example(question=q, answer=a['text'][0]).with_inputs("question")
           for eid, title, context, q, a in data]
    return exs

squad = load_dataset("squad")
squad_train = get_squad_split(squad, split="train")
squad_dev = get_squad_split(squad)
dev_exs = random.sample(squad_dev, k=20)

In [5]:
def get_squad_df(squad, split="validation", sample = False, sample_size = 50):
    data = squad[split]
    df = pd.json_normalize(data)
    if sample == True:
        df = df.sample(n=sample_size, random_state=1).reset_index(drop=True)
    df['answer'] = df['answers.text'].apply(lambda x: x[0])
    # df.rename({'answers.text': 'answer'}, axis=1, inplace=True)
    df = df[['context', 'question', 'answer']]
    return df

In [6]:
squad_open = get_squad_df(squad, 'train', True)
squad_close = get_squad_df(squad, 'train', True)
squad_open.head()

,context,question,answer
0,"On the next day, December 18, protests turned ...",How many people were estimated by authorities ...,"3,000"
1,Roman Catholicism was the sole established rel...,"Until the Reformation, what was the establishe...",Roman Catholicism
2,Israel retaliated against Egyptian shelling wi...,When did the war start up again?,March 1969
3,Zen Buddhist teaching is often full of paradox...,What Buddhist teachings are often full of para...,Zen
4,Sahih al-Bukhari narrates Muhammad describing ...,In which work did Welch express his belief tha...,Encyclopaedia of Islam


In [7]:
def get_halueval_split(dataset):
    data = zip(*[dataset['data'][field] for field in dataset['data'].features])
    exs = [dspy.Example(question=question, context=knowledge, right_answer=right_ans, hallucinated_answer=halu_ans).with_inputs("question")
           for knowledge, question, right_ans, halu_ans in data]
    return exs

halueval_qa = load_dataset('pminervini/HaluEval', 'qa')
halu_ds = get_halueval_split(halueval_qa)
halu_dev = random.sample(halu_ds, k=20)

In [8]:
len(halu_ds)

10000

In [9]:
def get_he_df(squad, split="data", sample = False, sample_size = 50):
    data = squad[split]
    df = pd.json_normalize(data)
    if sample == True:
        df = df.sample(n=sample_size, random_state=1).reset_index(drop=True)
    df = df[['knowledge', 'question', 'right_answer', 'hallucinated_answer']]
    return df

In [10]:
# he_close = get_he_df(halueval_qa, 'data', True)
he_open = get_he_df(halueval_qa, 'data', True)
he_open

,knowledge,question,right_answer,hallucinated_answer
0,Jack Elam is cast in occasional episodes as s...,Where did the actor who played sidekick Toothy...,the United States Navy,Jack Elam served in the Air Force.
1,"Loewald's father, who died shortly after his b...",What was the nationality of the man with whom ...,German,Hans Loewald studied philosophy with a French ...
2,The Knicks–Nuggets brawl was an on-court alter...,The Knicks-Nuggests brawl was the most penaliz...,"Auburn Hills, Michigan",The Knicks-Nuggets brawl was more infamous tha...
3,It was written by Paul McCartney (credited to...,"The song ""Your Mother Should Know"" is based on...",1961,"The song ""Your Mother Should Know"" is based on..."
4,"""The Adventures of Supergirl"" is the first epi...",ON what date did the man who played Michael Su...,"October 10, 2016",Tyler Hoechlin first appeared in The Adventure...
5,"Jules Sitruk (born April 16, 1990 in Lilas, ne...","What comedy film, written and directed by Gart...",Son of Rambow,Jules Sitruk did not act in any of Garth Jenni...
6,Non-Stop is a 2014 American mystery action thr...,Non-Stop starred the English actress best know...,Lady Mary Crawley,Michelle Dockery played Lady Mary.
7,Baby Blue is the fourth studio album by Mexica...,What is the is the fourth studio album by Mexi...,Baby Blue,"Anahí is married to Governor of Chiapas, Manue..."
8,"Homage to the Queen, Op. 42, by Malcolm Arnold...",What was the nationality of the costume design...,English,The costume designer for Homage to the Queen w...
9,Shortia is a small genus of subshrubs or peren...,"Which genus, Shortia or Schizophragma, has mor...",Shortia,Schizophragma has fewer species.


## Segmentation

In [11]:
from nltk.tokenize import sent_tokenize
import nltk

try:
    nltk.download("punkt", quiet=True)
except FileExistsError:  # multiprocessing race condition
    pass

In [12]:
class SentenceSplitter:
    def __init__(self):
        pass

    # use NLTK to split a text string into sentences for now
    def split_into_sentences(self, text):
        return sent_tokenize(text)

In [13]:
splitter = SentenceSplitter()

## Hallucination check

In [14]:
halu_check_model = models.HaluCheckRAG()
splitter = SentenceSplitter()

In [15]:
ex = halu_ds[0]
ex

Example({'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'context': "Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.First for Women is a woman's magazine published by Bauer Media Group in the USA.", 'right_answer': "Arthur's Magazine", 'hallucinated_answer': 'First for Women was started first.'}) (input_keys={'question'})

In [16]:
halu_check_model(question=ex['question'], response=ex['hallucinated_answer'], context=ex['context'])

True

In [17]:
import importlib
importlib.reload(models)

<module 'models' from '/home/michael/git/cs224u-final-project/models.py'>

In [18]:
modify_model = models.ModifyOpenQARAG()

In [19]:
modify_model(question=ex['question'],response=ex['hallucinated_answer'], context=ex['context'])

"Arthur's Magazine was started first."

In [20]:
lm.inspect_history(1)





The response to the question is wrong. Generate a correct answer to the question that is different from the given response and aligns with the provided context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Response: ${response}

Answer: Based on the context and taking the feedback on the response to the question into consideration, answer the question with short answers (limited to less than 6 words)

---

Context: Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.First for Women is a woman's magazine published by Bauer Media Group in the USA.

Question: Which magazine was started first Arthur's Magazine or First for Women?

Response: First for Women was started first.

Answer: Arthur's Magazine was started first.





In [31]:
records = []
index = -1
for ind in tqdm.tqdm(range(4000, len(halu_ds))):
    ex = halu_ds[ind]
    check_right_answer = halu_check_model(question=ex['question'], response=ex['right_answer'], context=ex['context'])
    check_hallucinated_answer = halu_check_model(question=ex['question'], response=ex['hallucinated_answer'], context=ex['context'])
    if check_right_answer is None:
        print(f"Wrong value for check_right_answer, index {ind}: answer {check_right_answer}")
    if check_hallucinated_answer  is None:
        print(f"Wrong value for check_hallucinated_answer, index {ind}: answer {check_hallucinated_answer}")
    if check_right_answer is not None:
        check_right_answer = int(check_right_answer)
    if check_hallucinated_answer is not None:
        check_hallucinated_answer = int(check_hallucinated_answer)
    index += 1
    records.append({
        'index': index,
        'original_index': ind,
        'label': 0,
        'prediction': check_right_answer
    })
    index += 1
    records.append({
        'index': index,
        'original_index': ind,
        'label': 1,
        'prediction': check_hallucinated_answer
    })
    if ind % 50 == 0:
        print(f'Checkpointing records with index {ind}')
        with open('records.pkl', 'wb') as file:
            pickle.dump(records, file)
df_results_halu_check = pd.DataFrame.from_records(records, index='index')
df_results_halu_check

  0%|          | 0/6000 [00:00<?, ?it/s]

Checkpointing records with index 4000
Wrong value for check_right_answer, index 4019: answer None


  1%|          | 50/6000 [00:31<1:30:41,  1.09it/s]

Checkpointing records with index 4050


  2%|▏         | 101/6000 [02:21<3:26:47,  2.10s/it]

...

Checkpointing records with index 9900


 99%|█████████▉| 5951/6000 [3:40:03<01:48,  2.21s/it]

Checkpointing records with index 9950


100%|██████████| 6000/6000 [3:41:50<00:00,  2.22s/it]


,original_index,label,prediction
index,,,
0,0,0,0.0
1,0,1,1.0
2,1,0,0.0
3,1,1,1.0
4,2,0,1.0
...,...,...,...
19996,9997,1,0.0
19997,9998,0,0.0
19998,9998,1,1.0


In [31]:
with open('records.pkl', 'rb') as file:
    records = pickle.load(file)
df_results_halu_check = pd.DataFrame.from_records(records, index='index')

In [36]:
# with open('df_results_halu_check.pkl', 'wb') as file:
#     pickle.dump(df_results_halu_check, file)

In [30]:
with open('df_results_halu_check.pkl', 'rb') as file:
    df_results_halu_check = pickle.load(file)

EOFError: Ran out of input

In [33]:
df_results_halu_check.dropna(inplace=True)

In [34]:
from sklearn.metrics import classification_report
classification_report(y_true=df_results_halu_check.loc[:,'label'], y_pred=df_results_halu_check.loc[:,'prediction'], output_dict=True)

{'0': {'precision': 0.7223054482377986,
  'recall': 0.9207035175879397,
  'f1-score': 0.809525913489153,
  'support': 9950.0},
 '1': {'precision': 0.8906747956214494,
  'recall': 0.6460301507537688,
  'f1-score': 0.7488786625502417,
  'support': 9950.0},
 'accuracy': 0.7833668341708543,
 'macro avg': {'precision': 0.806490121929624,
  'recall': 0.7833668341708542,
  'f1-score': 0.7792022880196974,
  'support': 19900.0},
 'weighted avg': {'precision': 0.806490121929624,
  'recall': 0.7833668341708543,
  'f1-score': 0.7792022880196974,
  'support': 19900.0}}

In [35]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_true=df_results_halu_check.loc[:,'label'], y_pred=df_results_halu_check.loc[:,'prediction']).ravel()
(tn, fp, fn, tp)

(9161, 789, 3522, 6428)